In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps

from config import g_key 

In [2]:
latlng_df = pd.DataFrame(pd.read_csv("CenPop2010CountyCenters.csv"))

latlng_df.head()

,STATEFP,COUNTYFP,COUNAME,STNAME,POPULATION,LATITUDE,LONGITUDE
0,6,1,Alameda,California,1510271,37.705201,-122.100393
1,6,3,Alpine,California,1175,38.728298,-119.847603
2,6,5,Amador,California,38091,38.394208,-120.775729
3,6,7,Butte,California,220000,39.669037,-121.707635
4,6,9,Calaveras,California,45578,38.163627,-120.633718


In [3]:
PCSA_df = pd.DataFrame(pd.read_csv("Primary_Care_Shortage_Areas_(PCSA).csv"))
PCSA_df.head() 

,OBJECTID,UNIT_COUNT,CNTY_FIPS,COUNTY,MSSA_ID,MSSA_NAME,DEFINITION,POPDENSITY,AREA_SQMI,POP,PCSA,PCSA_SCORE,PC_PHYS,PC_PHYS_R_CIV,POVScore,PHYSRatioScore,Shape__Area,Shape__Length
0,1,11,1,Alameda,1.1,Livermore Central and West/Spring Town,Urban,1752.000139,0.472102,43860,0,2,48.0,1186.104167,1,1,0.008283,0.489279
1,2,8,1,Alameda,1.2,Altamont/Livermore East/Midway/Mountain House/...,Rural,116.545153,1.950268,32480,1,6,12.0,3167.333333,1,5,0.086298,1.651976
2,3,12,7,Butte,10,Oroville/Palermo/Thermalito,Rural,175.468375,7.768868,42656,1,5,34.0,1408.352941,4,1,0.075224,1.615445
3,4,1,49,Modoc,100,Adin/Lookout/Newell,Frontier,2.315574,956.134376,1454,1,7,0.0,0.000000,2,5,0.267075,3.281548
4,5,2,51,Mono,102,Benton/Bridgeport/Chalfant/Crowley Lake/Colevi...,Frontier,1.957937,1452.267904,5343,1,6,1.0,5951.000000,1,5,0.813804,5.793963


In [4]:
#MSSA_NAME	text	MSSA_NAME	Name of the MSSA.
#Total_Population	numeric	Total_Population	Total Population of the MSSA.
#EST_Physicians	numeric	EST_Physicians	Estimated number of primary care physicians in the MSSA.  Includes MDs and DOs.
#EST_FNPPA	numeric	EST_FNPPA	Estimated number of family nurse practitioners and primary care physician assistants in the MSSA.
#EST_Providers	numeric	EST_Providers	Estimated number of primary care providers in the MSSA.  Calculated as EST_Physicians + (0.75 * EST_FNPPA).
#Provider_Ratio	numeric	Provider_Ratio	Population-to-provider ratio of the MSSA.  Calculated as Total_Population / EST_Providers.
#Score_Provider_Ratio	numeric	Score_Provider_Ratio	Subscore based on the provider ratio of the MSSA.  Possible values are 0-5.
#Pop_100FPL	numeric	Pop_100FPL	Population living  below the federal poverty level in the MSSA.
#PCT_100FPL	numeric	PCT_100FPL	Percent of the population in the MSSA living below the federal poverty level.  Calculated as Pop_100FPL / Total_Population.
#Score_Poverty	numeric	Score_Poverty	Subscore based on the poverty of the MSSA.  Possible values are 0-5.
#Score_Total	numeric	Score_Total	Final score of the MSSA for consideration as a Primary Care Shortage Area (PCSA).  Calculated as Score_Provider_Ratio + Score_Poverty.  Possible values are 0-10.
#PCSA	text	PCSA	Indicates whether the MSSA is considered a PCSA.  Possible values are Yes or No.
#Effective Date	timestamp	Effective_Date	Effective date of the PCSA designation.


In [5]:
PCSA_df.columns

Index(['OBJECTID', 'UNIT_COUNT', 'CNTY_FIPS', 'COUNTY', 'MSSA_ID', 'MSSA_NAME',
       'DEFINITION', 'POPDENSITY', 'AREA_SQMI', 'POP', 'PCSA', 'PCSA_SCORE',
       'PC_PHYS', 'PC_PHYS_R_CIV', 'POVScore', 'PHYSRatioScore', 'Shape__Area',
       'Shape__Length'],
      dtype='object')

In [6]:
PCSA_df.drop(columns=['OBJECTID','CNTY_FIPS','DEFINITION','MSSA_NAME','Shape__Area','UNIT_COUNT',
'Shape__Length','PC_PHYS_R_CIV'], inplace=True)
PCSA_df.head()

,COUNTY,MSSA_ID,POPDENSITY,AREA_SQMI,POP,PCSA,PCSA_SCORE,PC_PHYS,POVScore,PHYSRatioScore
0,Alameda,1.1,1752.000139,0.472102,43860,0,2,48.0,1,1
1,Alameda,1.2,116.545153,1.950268,32480,1,6,12.0,1,5
2,Butte,10,175.468375,7.768868,42656,1,5,34.0,4,1
3,Modoc,100,2.315574,956.134376,1454,1,7,0.0,2,5
4,Mono,102,1.957937,1452.267904,5343,1,6,1.0,1,5


In [ ]:
#dropped na

In [7]:
PCSA_df.dropna(inplace=True)

In [ ]:
#Merged my data frame with Lisa's demographic.csv file

In [8]:
PCSA_merged_df = PCSA_df.merge(latlng_df, how='outer', left_on=PCSA_df['COUNTY'] , right_on=latlng_df['COUNAME'])

In [9]:
PCSA_merged_df = PCSA_merged_df.groupby('COUNTY').agg({'POPDENSITY':'median', 'AREA_SQMI':'sum', 'POP':'sum', 
                                            'PCSA':'sum', 'PCSA_SCORE':'median', 'PC_PHYS':'sum',  
                                            'POVScore':'median', 'PHYSRatioScore':'median', 'LATITUDE':'median',
                                            'LONGITUDE': 'median'})
PCSA_merged_df.head(10)

,POPDENSITY,AREA_SQMI,POP,PCSA,PCSA_SCORE,PC_PHYS,POVScore,PHYSRatioScore,LATITUDE,LONGITUDE
COUNTY,,,,,,,,,,
Alameda,6432.968280,14.838507,1206808,6,2.0,2243.000000,1.5,0.5,37.705201,-122.100393
Alpine,1.567572,743.187747,1116,1,8.0,0.000000,3.0,5.0,38.728298,-119.847603
Amador,38.720054,347.749367,32435,1,4.0,37.000000,2.0,2.0,38.394208,-120.775729
Butte,57.138555,944.373893,188610,4,4.5,209.000000,3.0,1.5,39.669037,-121.707635
Calaveras,43.539210,102.578333,40020,0,4.0,26.000000,2.0,2.0,38.163627,-120.633718
Colusa,20.461334,1088.981974,9269,3,7.0,18.000000,2.0,5.0,39.156148,-122.085557
Contra Costa,1564.976169,21.166145,813373,4,3.0,1215.000000,1.0,1.0,37.935588,-122.030131
Del Norte,27.979075,805.684044,22835,0,4.0,29.000000,4.0,0.0,41.791232,-124.163088
El Dorado,173.074028,201.242189,159054,2,4.0,118.000000,2.0,3.0,38.746543,-120.744330


In [20]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

In [ ]:
#google maps

In [21]:
locations = PCSA_merged_df[["LATITUDE", "LONGITUDE"]].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights= PCSA_merged_df['PHYSRatioScore'],
                                 dissipating=False, max_intensity=8.5,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))